In [ ]:
#saving databases in faiss

import numpy as np
import os
import faiss
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
from sentence_transformers import SentenceTransformer


class EmbeddStoreFaissST:
    def __init__(self, model_name):
            self.model = SentenceTransformer(model_name,device = "cpu")

    def save_vectors_to_faiss_db(self, vectors, database_path):
        vectors_np = np.array(vectors, dtype=np.float32)
        index = faiss.IndexFlatIP(vectors_np.shape[1])
        faiss.normalize_L2(vectors_np)
        index.add(vectors_np)
        os.makedirs(os.path.dirname(database_path), exist_ok=True)
        try:
            faiss.write_index(index, database_path)
            print(f"Faiss database saved successfully to {database_path}")
        except Exception as e:
            print(f"Error saving Faiss database: {e}")

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()


    def load_faissdb(db_path, user_input, segmented_text):
        index = faiss.read_index(db_path)
        input_text_vector = self.encode_text(user_input)
        input_text_vector = np.array(input_text_vector, dtype=np.float32)
        distances, indices = index.search(np.array(input_text_vector), k=5)
        similar_texts = [segmented_text[i] for i in indices[0]]
        return similar_texts

    def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks

with open('ribbon_global.txt', 'r', encoding='utf-8') as file:
    file_contents = file.read()

segmented_text = EmbeddStoreFaissST.segment_text(file_contents, chunk_size=50)


model = EmbeddStoreFaissST(model_name='sentence-transformers/all-mpnet-base-v2')


database_path = "vector_db_test_sd/faiss_store/ribbon_global_1" 
content_embedded = model.encode_text(text=segmented_text)
model.save_vectors_to_faiss_db(content_embedded, database_path)


In [ ]:
#retrieval using faiss

import requests
import numpy as np
import faiss
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
from sentence_transformers import SentenceTransformer

import requests
class RetrivalFaissST:
    def __init__(self, model_name):
            self.model = SentenceTransformer(model_name,device = "cpu")

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()

    @classmethod
    def load_faissdb(cls, db_path, user_input, segmented_text, input_text_vector):
        index = faiss.read_index(db_path)
        input_text_vector = np.array(input_text_vector, dtype=np.float32)
        distances, indices = index.search(input_text_vector.reshape(1, -1), k=3)
        similar_texts = [segmented_text[i] for i in indices[0]]
        return similar_texts
        

    def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks


with open('ribbon_global.txt', 'r', encoding='utf-8') as file:
    file_contents = file.read()

segmented_text = RetrivalFaissST.segment_text(file_contents, chunk_size=50)

model = RetrivalFaissST(model_name='sentence-transformers/all-mpnet-base-v2')

user_input = input("User input: ")

database_path1 = "vector_db_test_sd/faiss_store/ribbon_global_1" 
input_text_vector = model.encode_text(user_input)
similar_texts = model.load_faissdb(database_path1,user_input,segmented_text,input_text_vector)
context= similar_texts[0]


print(context)

# data = set(context)
final_context = " ".join(context)
print(final_context)
print("##################################################################################")

payload = {
  "message_content": user_input,
  "content": final_context,
  "company_name": "ribbon_global",
  "domain": "global services"
}

response = requests.post("http://46.43.144.145:7010/rulebased_bot",json=payload)

print(response.json())